### Premier League data Plotted using APis
 Using an api from football-data.org,
 with auth token: de27c6f949e040418ee4fcb5206ca657
 I have plotted data from that api, getting tables and sorting them

In [ ]:
import requests
url = "https://api.football-data.org/v4/competitions/PL/matches"
headers = {"X-Auth-Token": "de27c6f949e040418ee4fcb5206ca657"}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}, {response.text}")

In [ ]:
import pandas as pd
df = pd.DataFrame(data['matches'])

df


 Selecting the name of a team alone from the "homeTeam" data

In [ ]:
df['home_team_name'] = df['homeTeam'].apply(lambda x: x['name'] if isinstance(x, dict) else None)
df['away_team_name'] = df['awayTeam'].apply(lambda x: x['name'] if isinstance(x, dict) else None)

df['home_goals'] = df['score'].apply(lambda x: x['fullTime']['home'] if isinstance(x, dict) and 'fullTime' in x else None)
df['away_goals'] = df['score'].apply(lambda x: x['fullTime']['away'] if isinstance(x, dict) and 'fullTime' in x else None)


df['utcDate'] = pd.to_datetime(df['utcDate'])

# Format as 'Day, Month Date, Year - HH:MM AM/PM'
df['match_date'] = df['utcDate'].dt.strftime('%A, %B %d, %Y - %I:%M %p')


Plotting the matches in order of most recent

In [ ]:

df_pl = df[df['status'] == 'FINISHED'].copy()

df_pl= df_pl[['home_team_name','status', 'away_team_name', 'home_goals', 'away_goals','match_date']]

df_pl['match_date'] = pd.to_datetime(df_pl['match_date'], format='%A, %B %d, %Y - %I:%M %p')
df_pl = df_pl.sort_values(by='match_date', ascending=False)

df_pl.head(10)


### To Get Standings 
to get the top 10 on the Premier League table

In [ ]:
import requests
url = "https://api.football-data.org/v4/competitions/PL/standings"
headers = {"X-Auth-Token": "de27c6f949e040418ee4fcb5206ca657"}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}, {response.text}")
# Extract standings data
standings = data['standings'][0]['table']  # The first element contains league table data

# Convert to DataFrame
df = pd.DataFrame(standings)


In [ ]:
df['team_name'] = df['team'].apply(lambda x: x['name'] if isinstance(x, dict) else None)


df_standings= df[['position','team_name','playedGames','won','draw','lost','points','goalsFor','goalsAgainst','goalDifference']]

df_standings.head(10)